In [1]:
from rouge_score import rouge_scorer
from itertools import combinations
from nltk.tokenize import word_tokenize
import json
import pandas as pd

In [2]:
def load_jsonl(data_path):
    data = []
    with open(data_path) as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [3]:
data = load_jsonl("reddit_data/reddit_cands_100.jsonl")

In [4]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [5]:
for entry in data:
    cands = list(combinations(entry['idx'], 2))
    cands += list(combinations(entry['idx'], 3))
    cands = [' '.join(tup) for tup in cands]
    scores = []
    for cand in cands:
        rouge_scores = scorer.score(cand, entry['summary'])
        score = sum([s.fmeasure for s in rouge_scores.values()]) / 3
        scores.append((cand, score))
    scores.sort(key=lambda x : x[1], reverse=True)
    entry['scores'] = scores

In [6]:
df_temp = pd.json_normalize(data)

In [7]:
df_temp

,text,summary,idx,scores
0,this actually happened a couple of years ago. ...,confuse a 5th grade girl for a boy in front of...,[i grew up in germany where i went to a german...,[(i grew up in germany where i went to a germa...
1,"it was last october, but i'm feeling the fall-...","i found my estranged dad, thought i loved him ...","[it was last october, but i'm feeling the fall...",[(during the most acute months of grieving for...
2,so i had the brilliant idea to use veet hair r...,had my balls burned by sauron and was left dev...,[so i had the brilliant idea to use veet hair ...,[(the slight peroxide kinda smell ensued and a...
3,today i was going to have a bath after a long ...,peppermint + bath = burning cold ladybits.,[today i was going to have a bath after a long...,[(today i was going to have a bath after a lon...
4,"i haven't had a bath in practically years so, ...","got too high and too hot in the bath, almost c...","[i haven't had a bath in practically years so,...","[(picture this; a very cramped bathroom, no wi..."
...,...,...,...,...
95,a bit of background: i'm almost certain that i...,pooped and peed all over a porta-potty while t...,[a bit of background: i'm almost certain that ...,"[(as i begin, as my friend once put it, ""choco..."
96,"average day at work, not too busy, not too slo...","turn around for a split second, little girl go...","[average day at work, not too busy, not too sl...",[(i'm getting ready to go on my break while wa...
97,so my day pretty much sucked.\nlet's start off...,"i ate a wendy's frosty, got the shits, burned ...",[let's start off with the shitty morning of me...,[(we drove to wendy's and ordered four chocola...
98,yesterday evening my mother and i were walking...,"didn't sit down while taking off my shoes, got...",[yesterday evening my mother and i were walkin...,"[(for some odd reason, i didn't sit down on th..."


In [8]:
df_temp.to_json("reddit_data/reddit_rouge_100.jsonl", orient='records', lines=True)